In [1]:
import FlowCal
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import os
import pandas as pd

In [ ]:
# path to fcs files
path = 'fox'

# create empty lists for storing data
data_norm1 = []; data_norm2 = []
data_stdrd1 = []; data_stdrd2 = []
samples = []

# channels_fox_sox = ['FSC-A','SSC-A','FL1-A','FL7-A']
# channels_oct = ['FSC-A','SSC-A','FL1-A','FL13-A']
channels = ['FSC-A','SSC-A','FL1-A','FL2-A','FL3-A','FL4-A','FL5-A','FL6-A','FL7-A','FL8-A','FL9-A','FL10-A','FL11-A','FL12-A','FL13-A','FL14-A']

ctrl_sox = 0
ctrl_oct = 0

# load files
for fox_file in os.listdir(path):
    fox = FlowCal.io.FCSData(path + '/' + fox_file)
    fox = fox[:,channels]

    # find corresponding single cell masks in folder "robust_clust_fox/singlets masks"
    exp = fox_file.split(' DE')[0]
    if len(exp.split(' ')) > 1:
        exp = exp.split(' ')[1]

    if len(fox) < 2000:
        print('less than 2000 events in fox')
        continue

    for sox_file in os.listdir('sox'):
        if exp in sox_file:
            sox = FlowCal.io.FCSData('sox/' + sox_file)
            sox = sox[:,channels]
            ctrl_sox = 1
            break
    
    # if no sox file, skip
    if ctrl_sox == 0:
        print('no sox file')
        continue
    ctrl_sox = 0

    if len(sox) < 2000:
        print('less than 2000 events in sox')
        continue

    for oct_file in os.listdir('oct'):
        if exp in oct_file:
            oct = FlowCal.io.FCSData('oct/' + oct_file)
            oct = oct[:,channels]
            ctrl_oct = 1
            break
    
    # if no oct file, skip
    if ctrl_oct == 0:
        print('no oct file')
        continue
    ctrl_oct = 0

    if len(oct) < 2000:
        print('less than 2000 events in oct')
        continue

    # apply log transform
    fox[fox < 0] = 0; fox = np.log10(fox+1)
    sox[sox < 0] = 0; sox = np.log10(sox+1)
    oct[oct < 0] = 0; oct = np.log10(oct+1)

    # fox_min = np.min(fox, axis=0); fox_min[fox_min > 0] = 0; fox = fox - fox_min; fox[:,2:] = np.log10(fox[:,2:]+1)
    # sox_min = np.min(sox, axis=0); sox_min[sox_min > 0] = 0; sox = sox - sox_min; sox[:,2:] = np.log10(sox[:,2:]+1)
    # oct_min = np.min(oct, axis=0); oct_min[oct_min > 0] = 0; oct = oct - oct_min; oct[:,2:] = np.log10(oct[:,2:]+1)

    # clip to [0, 0.95] quantile
    quantiles = np.quantile(fox, 0.85, axis=0); fox_clipped = np.clip(fox, 0, quantiles)
    quantiles = np.quantile(sox, 0.85, axis=0); sox_clipped = np.clip(sox, 0, quantiles)
    quantiles = np.quantile(oct, 0.85, axis=0); oct_clipped = np.clip(oct, 0, quantiles)
    # fox_clipped = np.clip(fox, 0, quantiles[0])
    # sox_clipped = np.clip(sox, 0, quantiles[1])
    # oct_clipped = np.clip(oct, 0, quantiles[2])

    # apply min-max normalization
    fox_norm = MinMaxScaler().fit_transform(fox_clipped)
    sox_norm = MinMaxScaler().fit_transform(sox_clipped)
    oct_norm = MinMaxScaler().fit_transform(oct_clipped)
    # fox_norm = (fox_clipped - extrema[0,0]) / (extrema[0,1] - extrema[0,0])
    # sox_norm = (sox_clipped - extrema[1,0]) / (extrema[1,1] - extrema[1,0])
    # oct_norm = (oct_clipped - extrema[2,0]) / (extrema[2,1] - extrema[2,0])

    # apply standardization
    fox_stdrd = StandardScaler().fit_transform(fox)
    sox_stdrd = StandardScaler().fit_transform(sox)
    oct_stdrd = StandardScaler().fit_transform(oct)
    # fox_stdrd = (fox - stats[0,0]) / stats[0,1]
    # sox_stdrd = (sox - stats[1,0]) / stats[1,1]
    # oct_stdrd = (oct - stats[2,0]) / stats[2,1]
    # fox_stdrd = (fox_clipped - stats[0,0]) / stats[0,1]
    # sox_stdrd = (sox_clipped - stats[1,0]) / stats[1,1]
    # oct_stdrd = (oct_clipped - stats[2,0]) / stats[2,1]

    # randomly select 1000 events
    order = np.arange(len(fox)); np.random.shuffle(order); fox_norm = fox_norm[order[:2000],:]; fox_stdrd = fox_stdrd[order[:2000],:]
    order = np.arange(len(sox)); np.random.shuffle(order); sox_norm = sox_norm[order[:2000],:]; sox_stdrd = sox_stdrd[order[:2000],:]
    order = np.arange(len(oct)); np.random.shuffle(order); oct_norm = oct_norm[order[:2000],:]; oct_stdrd = oct_stdrd[order[:2000],:]

    # build matrix of shape fox_norm.shape[0] + sox_norm.shape[0] + oct_norm.shape[0], fox_norm.shape[1] + sox_norm.shape[1] + oct_norm.shape[1]
    # fill with -1
    # fill diagonal blocks with fox_norm, sox_norm, oct_norm
    # fill off-diagonal blocks with -1
    # super_norm1 = np.zeros((6000, 12)); super_norm1.fill(-1)
    # super_norm1[:fox_norm.shape[0],:fox_norm.shape[1]] = fox_norm
    # super_norm1[fox_norm.shape[0]:fox_norm.shape[0]+sox_norm.shape[0],fox_norm.shape[1]:fox_norm.shape[1]+sox_norm.shape[1]] = sox_norm
    # super_norm1[fox_norm.shape[0]+sox_norm.shape[0]:,fox_norm.shape[1]+sox_norm.shape[1]:] = oct_norm

    # super_norm2 = np.zeros((6000, 5)); super_norm2.fill(-1)
    # super_norm2[:fox_norm.shape[0],:fox_norm.shape[1]] = fox_norm
    # super_norm2[fox_norm.shape[0]:fox_norm.shape[0]+sox_norm.shape[0],:sox_norm.shape[1]] = sox_norm
    # super_norm2[fox_norm.shape[0]+sox_norm.shape[0]:,:oct_norm.shape[1]-1] = oct_norm[:,:-1]
    # super_norm2[fox_norm.shape[0]+sox_norm.shape[0]:,-1] = oct_norm[:,-1]
    super_norm1 = np.zeros((6000, len(channels)*3)); super_norm1.fill(-1)
    super_norm1[:fox_norm.shape[0],:fox_norm.shape[1]] = fox_norm
    super_norm1[fox_norm.shape[0]:fox_norm.shape[0]+sox_norm.shape[0],fox_norm.shape[1]:fox_norm.shape[1]+sox_norm.shape[1]] = sox_norm
    super_norm1[fox_norm.shape[0]+sox_norm.shape[0]:,fox_norm.shape[1]+sox_norm.shape[1]:] = oct_norm

    # super_norm2 = np.zeros((6000, len(channels))); super_norm2.fill(-1)
    # super_norm2[:fox_norm.shape[0],:] = fox_norm
    # super_norm2[fox_norm.shape[0]:fox_norm.shape[0]+sox_norm.shape[0],:] = sox_norm
    # super_norm2[fox_norm.shape[0]+sox_norm.shape[0]:,:] = oct_norm

    # super_stdrd1 = np.zeros((6000, 12))
    # super_stdrd1[:fox_stdrd.shape[0],:fox_stdrd.shape[1]] = fox_stdrd
    # super_stdrd1[fox_stdrd.shape[0]:fox_stdrd.shape[0]+sox_stdrd.shape[0],fox_stdrd.shape[1]:fox_stdrd.shape[1]+sox_stdrd.shape[1]] = sox_stdrd
    # super_stdrd1[fox_stdrd.shape[0]+sox_stdrd.shape[0]:,fox_stdrd.shape[1]+sox_stdrd.shape[1]:] = oct_stdrd

    # super_stdrd2 = np.zeros((6000, 5))
    # super_stdrd2[:fox_stdrd.shape[0],:fox_stdrd.shape[1]] = fox_stdrd
    # super_stdrd2[fox_stdrd.shape[0]:fox_stdrd.shape[0]+sox_stdrd.shape[0],:sox_stdrd.shape[1]] = sox_stdrd
    # super_stdrd2[fox_stdrd.shape[0]+sox_stdrd.shape[0]:,:oct_stdrd.shape[1]-1] = oct_stdrd[:,:-1]
    # super_stdrd2[fox_stdrd.shape[0]+sox_stdrd.shape[0]:,-1] = oct_stdrd[:,-1]

    super_stdrd1 = np.zeros((6000, len(channels)*3))
    super_stdrd1[:fox_stdrd.shape[0],:fox_stdrd.shape[1]] = fox_stdrd
    super_stdrd1[fox_stdrd.shape[0]:fox_stdrd.shape[0]+sox_stdrd.shape[0],fox_stdrd.shape[1]:fox_stdrd.shape[1]+sox_stdrd.shape[1]] = sox_stdrd
    super_stdrd1[fox_stdrd.shape[0]+sox_stdrd.shape[0]:,fox_stdrd.shape[1]+sox_stdrd.shape[1]:] = oct_stdrd

    # super_stdrd2 = np.zeros((6000, len(channels)))
    # super_stdrd2[:fox_stdrd.shape[0],:] = fox_stdrd
    # super_stdrd2[fox_stdrd.shape[0]:fox_stdrd.shape[0]+sox_stdrd.shape[0],:] = sox_stdrd
    # super_stdrd2[fox_stdrd.shape[0]+sox_stdrd.shape[0]:,:] = oct_stdrd

    # append data and labels
    data_norm1.append(super_norm1)
    # data_norm2.append(super_norm2)
    data_stdrd1.append(super_stdrd1)
    # data_stdrd2.append(super_stdrd2)
    # data_stdrd.append(sc_stdrd)
    samples.append(exp)

    print('Finished ' + exp)

# convert to numpy arrays
data_norm1 = np.array(data_norm1)
# data_norm2 = np.array(data_norm2)
data_stdrd1 = np.array(data_stdrd1)
# data_stdrd2 = np.array(data_stdrd2)
# data_stdrd = np.array(data_stdrd)
samples = np.array(samples)

In [8]:
# load annotations from labels.csv
annotations = pd.read_csv('labels.csv', delimiter=';', decimal=',')

# create pandas dataframe from labels
df = pd.DataFrame(samples, columns=['sample'])

# now join annotations and labels
df = pd.merge(df, annotations, on='sample')

# replace yes/no with 1/0
df[['D4 pass', 'D25 pass']] = df[['D4 pass', 'D25 pass']].replace({'yes': 1, 'no': 0})

In [3]:
# save data and labels
np.save('train-test data/supermatrix_unfiltered_all_log_clipped_0-1_norm1', data_norm1)
# np.save('train-test data/supermatrix_unfiltered_all_log_clipped_0-1_norm2', data_norm2)
# np.save('train-test data/supermatrix_unfiltered_all_log_stdrd1', data_stdrd1)
# np.save('train-test data/supermatrix_unfiltered_all_log_stdrd2', data_stdrd2)
# df.to_csv('annotations.csv', index=False, sep=';')

In [ ]:
# path to fcs files
path = 'D13 Ida'

# create empty lists for storing data
data_norm = []
data_stdrd = []
samples = []

channels = ['FSC-A','SSC-A','FL1-A','FL2-A','FL3-A','FL4-A','FL5-A','FL6-A','FL7-A','FL8-A','FL9-A','FL10-A','FL11-A','FL12-A','FL13-A','FL14-A']

ctrl_sox = 0
ctrl_oct = 0

# load files
for file in os.listdir(path):
    s = FlowCal.io.FCSData(path + '/' + file)
    s = s[:,channels]

    # find corresponding single cell masks in folder "robust_clust_fox/singlets masks"
    exp = file.split(' D13')[0]

    if len(s) < 1000:
        print('less than 1000 events')
        continue

    # apply log transform
    s[s < 0] = 0; s = np.log10(s+1)

    # clip to [0, 0.95] quantile
    quantiles = np.quantile(s, 0.85, axis=0); s_clipped = np.clip(s, 0, quantiles)

    # apply min-max normalization
    s_norm = MinMaxScaler().fit_transform(s_clipped)

    # apply standardization
    s_stdrd = StandardScaler().fit_transform(s)

    # randomly select 1000 events
    order = np.arange(len(s)); np.random.shuffle(order); s_norm = s_norm[order[:1000],:]; s_stdrd = s_stdrd[order[:1000],:]

    # append data and labels
    data_norm.append(s_norm)
    data_stdrd.append(s_stdrd)
    samples.append(exp)

    print('Finished ' + exp)

# convert to numpy arrays
data_norm = np.array(data_norm)
data_stdrd = np.array(data_stdrd)
samples = np.array(samples)

# load annotations from labels.csv
annotations = pd.read_csv('labels.csv', delimiter=';', decimal=',')

# create pandas dataframe from labels
df = pd.DataFrame(samples, columns=['sample'])

# now join annotations and labels
df = pd.merge(df, annotations, on='sample')

# replace yes/no with 1/0
df[['D4 pass', 'D25 pass']] = df[['D4 pass', 'D25 pass']].replace({'yes': 1, 'no': 0})

In [20]:
# save data and labels
np.save('train-test data/d13_unfiltered_all_log_stdrd', data_stdrd)
np.save('train-test data/d13_unfiltered_all_log_norm', data_norm)
df.to_csv('d13_annotations.csv', index=False, sep=';')

Misclassifications revisited

In [ ]:
import umap

# load data
annotations = pd.read_csv('supermatrix_labels.csv', delimiter=';')
samples = annotations['sample'].values

# of each sample, take only the first 2000 events in columns 0-15
data_fox = data_stdrd1[:,:2000,:16]
data_sox = data_stdrd1[:,2000:4000,16:32]
data_oct = data_stdrd1[:,4000:,32:]

# multiply each entry in labels by 2000
names = np.repeat(annotations['sample'].values, 2000, axis=0)
labels = np.repeat(annotations['D4 pass'].values, 2000, axis=0)

# apply umap
reducer = umap.UMAP(random_state=41)
embedding_fox = reducer.fit_transform(data_fox.reshape(-1, data_fox.shape[2]))
reducer = umap.UMAP(random_state=42)
embedding_sox = reducer.fit_transform(data_sox.reshape(-1, data_sox.shape[2]))
reducer = umap.UMAP(random_state=43)
embedding_oct = reducer.fit_transform(data_oct.reshape(-1, data_oct.shape[2]))

test_folds = np.load('test_folds_super.npy', allow_pickle=True)
train_folds = np.load('train_folds_super.npy', allow_pickle=True)

test_set = test_folds[6]
samples_train = samples[np.isin(samples, test_set, invert=True)]

palette1 = [sns.color_palette("Blues")[-1], sns.color_palette("Blues")[1]]
palette2 = [sns.color_palette("tab10")[3]]  

for sample in samples_train:
    fig, ax = plt.subplots(1, 3, figsize=(15,5)) 
    idx = np.isin(names, test_set, invert=True) & np.isin(names, sample, invert=True)
    sns.scatterplot(x=embedding_fox[idx,0], y=embedding_fox[idx,1], hue=labels[idx], palette=palette1, s=1.5, ax=ax[0])
    sns.scatterplot(x=embedding_fox[names == sample,0], y=embedding_fox[names == sample,1], hue=names[names == sample], palette=['red'], s=1.5, ax=ax[0]) 
    ax[0].set_xticks([])
    ax[0].set_yticks([])
    ax[0].set_title('FOXA2')
    sns.scatterplot(x=embedding_sox[idx,0], y=embedding_sox[idx,1], hue=labels[idx], palette=palette1, s=1.5, ax=ax[1])
    sns.scatterplot(x=embedding_sox[names == sample,0], y=embedding_sox[names == sample,1], hue=names[names == sample], palette=['red'], s=1.5, ax=ax[1])
    ax[1].set_xticks([])
    ax[1].set_yticks([])
    ax[1].set_title('SOX17')
    ax[1].legend([],[], frameon=False)
    sns.scatterplot(x=embedding_oct[idx,0], y=embedding_oct[idx,1], hue=labels[idx], palette=palette1, s=1.5, ax=ax[2])
    sns.scatterplot(x=embedding_oct[names == sample,0], y=embedding_oct[names == sample,1], hue=names[names == sample], palette=['red'], s=1.5, ax=ax[2])
    ax[2].set_xticks([])
    ax[2].set_yticks([])
    ax[2].set_title('OCT4')
    ax[2].legend([],[], frameon=False)
    plt.suptitle(f'UMAP projection of {sample} against training batch')
    ground_truth = annotations[annotations['sample'] == sample]['D4 pass'].values[0]
    plt.savefig(f'umap_stdrd_train6/{sample}_{ground_truth}_train6.png')
    plt.show()

In [64]:
test_folds = np.load('test_folds_super.npy', allow_pickle=True)
train_folds = np.load('train_folds_super.npy', allow_pickle=True)

In [ ]:
test_set = test_folds[1]
train_indices = np.isin(names, test_set, invert=True)
print(test_set)

train_fold = data[np.isin(samples, test_set, invert=True),:,:]
test_fold = data[np.isin(samples, test_set),:,:]
samples_train = samples[np.isin(samples, test_set, invert=True)]
samples_test = samples[np.isin(samples, test_set)]

train_fold_p = train_fold[annotations['D4 pass'].values[np.isin(samples, test_set, invert=True)] == 1]
train_fold_n = train_fold[annotations['D4 pass'].values[np.isin(samples, test_set, invert=True)] == 0]

In [ ]:
# plot
exp = '587'
palette1 = [sns.color_palette("Blues")[-1], sns.color_palette("Blues")[1]]
palette2 = [sns.color_palette("tab10")[3]]
# sns.scatterplot(x=embedding[train_indices,0], y=embedding[train_indices,1], hue=labels[train_indices], palette=palette1, s=1.5)
# sns.scatterplot(x=embedding[names == exp,0], y=embedding[names == exp,1], hue=names[names == exp], palette=['red'], s=1.5)   
# plt.legend()
# plt.title(f'UMAP projection of {exp} against training batch')
# plt.show()

fig, ax = plt.subplots(1, 3, figsize=(15,5)) 
sns.scatterplot(x=embedding_fox[train_indices,0], y=embedding_fox[train_indices,1], hue=labels[train_indices], palette=palette1, s=1.5, ax=ax[0])
sns.scatterplot(x=embedding_fox[names == exp,0], y=embedding_fox[names == exp,1], hue=names[names == exp], palette=['red'], s=1.5, ax=ax[0]) 
ax[0].set_xticks([])
ax[0].set_yticks([])
ax[0].set_title('FOXA2')
sns.scatterplot(x=embedding_sox[train_indices,0], y=embedding_sox[train_indices,1], hue=labels[train_indices], palette=palette1, s=1.5, ax=ax[1])
sns.scatterplot(x=embedding_sox[names == exp,0], y=embedding_sox[names == exp,1], hue=names[names == exp], palette=['red'], s=1.5, ax=ax[1])
ax[1].set_xticks([])
ax[1].set_yticks([])
ax[1].set_title('SOX17')
ax[1].legend([],[], frameon=False)
sns.scatterplot(x=embedding_oct[train_indices,0], y=embedding_oct[train_indices,1], hue=labels[train_indices], palette=palette1, s=1.5, ax=ax[2])
sns.scatterplot(x=embedding_oct[names == exp,0], y=embedding_oct[names == exp,1], hue=names[names == exp], palette=['red'], s=1.5, ax=ax[2])
ax[2].set_xticks([])
ax[2].set_yticks([])
ax[2].set_title('OCT4')
ax[2].legend([],[], frameon=False)
plt.suptitle(f'UMAP projection of {exp} against training batch')
plt.savefig(f'umaps_{exp}_train6.png')
plt.show()

# # create 2x2 seaborn plot
# fig, axs = plt.subplots(2, 2, figsize=(10,10))
# for idx, channel in enumerate([0,1,2,14]):
#     sns.kdeplot(train_fold_n.reshape(-1, train_fold.shape[2])[:,channel], color=palette1[0], ax=axs[idx//2,idx%2], label='0')
#     # sns.histplot(train_fold_n.reshape(-1, train_fold.shape[2])[:,channel], kde=True, stat='density', bins=100, element="step", fill=False, color=palette1[0], ax=axs[idx//2,idx%2], label='0')
#     sns.kdeplot(train_fold_p.reshape(-1, train_fold.shape[2])[:,channel], color=palette1[1], ax=axs[idx//2,idx%2], label='1')
#     # sns.histplot(train_fold_p.reshape(-1, train_fold.shape[2])[:,channel], kde=True, stat='density', bins=100, element="step", fill=False, color=palette1[1], ax=axs[idx//2,idx%2], label='1')
#     sns.kdeplot(test_fold[samples_test == exp].reshape(-1, test_fold.shape[2])[:,channel], color=palette2[0], ax=axs[idx//2,idx%2], label=exp)
#     # sns.histplot(test_fold[samples_test == exp].reshape(-1, test_fold.shape[2])[:,channel], kde=True, stat='density', bins=100, element="step", fill=False, color=palette2[0], ax=axs[idx//2,idx%2], label=exp)
#     axs[idx//2,idx%2].legend()
#     axs[idx//2,idx%2].set_title(f'{channels[channel]}')
# plt.show()